# Classical Phase Estimation (with QFT) module

Present notebook reviews the classical **Quantum Phase Estimation Algorithm**, which using the inverse of **Quantum Fourier Transform** ($\mathcal{QFT}^{\dagger}$), allows the estimation of the autovalues (phases) of an unitary operator. From now we will call this algorithm **CQPE** from now. The **CQPE** was developed into module *classical_qpe.py* of the package *PE*  of present library *QQuantLib* (**QQuantLib/PE/classical_pe.py**). 


This algorithm was the developed as a python class called: *CQPE* inside the **QQuantLib/PE/classical_qpe.py**.

Present notebook and module are based on the following references:

* *Brassard, G., Hoyer, P., Mosca, M., & Tapp, A. (2000). Quantum amplitude amplification and estimation.AMS Contemporary Mathematics Series, 305. https://arxiv.org/abs/quant-ph/0005055v1*
* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*


In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Initial Inputs

For using the *ClassicalPE* python class inside the **QQuantLib/PE/classical_pe** module 2 mandatory inputs should be provided:

* 1. Initial State: this will be the initial quantum state needed for applying the Unitary Operator.
* 2. Unitary operator: the operator whose phase we want to estimate.

For explain how the *ClassicalPE* class works we are going to use the **IQPE** example from Qiskit textbook:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb

We are going to reproduce the section **IPE example with a 1-qubit gate for U** from Qiskit example.

In this section we are going to create this two mandatory inputs!

In [ ]:
#Number Of Qbits
n_qbits = 1

### 1.1 Initial State

Initial State can be:
1. QLM QRoutine
2. QLM gate (or abstract gate)

In the qiskit example the initial state will be $|1\rangle$. 

Following cell creates this initial state:

In [ ]:
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

### 1.2 Unitary operator.

Unitary operator can be:

1. QLM QRoutine
2. QLM gate (or abstract gate)

In the qiskit example the unitary operator is the $\mathcal{S}$ gate. So the application of the unitary operator over the initial state will be:

$$\mathcal{S}|1\rangle = e^{i\frac{\pi}{2}}|1\rangle$$

In [ ]:
unitary_operator = qlm.PH(np.pi/2.0)  

In [ ]:
%qatdisplay unitary_operator --svg

## 2. Class CQPE: classical Quantum Phase Estimation algorithm step by step 

The problem of phase estimation can be stated as follows. Given an initial state $\left|\Psi \right\rangle$ and a phase operator $\mathcal{P}$ such that:

$$\mathcal{P}\left|\Psi \right\rangle = e^{2\pi i\lambda}\left|\Psi \right\rangle,$$

our goal is estimating $\lambda$.

So far we have the initial state $\left|\Psi \right\rangle = |1\rangle$ and the unitary operator whose phase we want to estimate $\mathcal{P} = \mathcal{S}$. In this section we are going to describe the class step by step and explain the basics of the **CQPE** algorithm

### 2.1 Calling the *CQPE* class

The *CQPE* is inside **QQuantLib/PE/classical_qpe** module. 

In order to instantiate the class we need to provide a pyhton dictionary. Mandatory keys that user should provide are, as explained in section 1:

* initial_state : QLM routine or gate with an initial state $|\Psi\rangle$ was loaded (created in Section 1).
* unitary_operator :  QLM gate or routine with an Unitary operator ready for be applied to initial state $|\Psi\rangle$ (created in Section 1)

Additionally there are other keys that are important in order for configurate the method:

* auxiliar_qbits_number : int. Number of auxiliar qbits used for phase estimation (default 8).
* qpu : QLM solver. If not provided class try to creates a PyLinalg solver. It is recomended give this key to the class.
* shots : int number of shots for quantum job (default 10).

In [ ]:
#Load Class
from QQuantLib.PE.classical_qpe import CQPE

In [ ]:
auxiliar_qbits_number = 2
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : auxiliar_qbits_number,
}
qft_pe = CQPE(**qft_pe_dict)

When the class is instantiated the properties *initial_state* and *q_gate* are overwritten with the given keys **initial_state** and **unitary_operator** respectively

In [ ]:
c = qft_pe.initial_state
%qatdisplay c --svg

In [ ]:
a = qft_pe.q_gate
%qatdisplay a --depth 2 --svg

### 2.2 Classical Quantum Phase Estimation algorithm: step by step

Now we are going to review step by step the **CQPE** algorithm using different programed methods of the **CQPE** class

#### 2.2.1. Initialize the quantum program.

First thing is calling the method **init_pe**. Following actions are done by this method:
1. Creation of QLM program from *initial_state* QLM routine (or AbstractGate). The QLM program is stored in *q_prog* property.
2. Allocation of the auxiliar qbits mandatory for the **CQPE** algorithm. It is stored in the *q_aux* property.

In [ ]:
#Initialize the quantum program
qft_pe.init_pe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliar qbit bits was allocated
circuit = qft_pe.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 0 --svg

#### 2.2.2. Classical Quantum Phase Estimation (with QFT) Algorithm:

We are going to decomposed the **CQPE** algorithm in 2 parts.

1. Application of controlled powers of the Unitary Operator
2. Application of the inverse of the *QFT*.

#### Application of controlled powers of the Unitary Operator

The first part will done following steps:
1. Apply a Haddamard gate to each auxiliar qbit
2. Each auxiliar qbit will appply a controlled power of the unitay operator to the initial state.


This can be done by calling the *apply_controlled_operations* method with following arguments:

* Quantum Program with initial_state
* Quantum Routine or AbstractGate with phase operator $\mathcal{P}$
* Auxiliars Qbits

This methods return the quantum program with the operations explained in this part.

In [ ]:
prog = qft_pe.apply_controlled_operations(qft_pe.q_prog, qft_pe.q_gate, qft_pe.q_aux)

In [ ]:
c = prog.to_circ()
%qatdisplay c --depth 1 --svg

#### Application of the inverse of the *QFT*

In this part of the algorithm the $\mathcal{QFT}^{\dagger}$ will be applied to the auxiliar qbits.

This is done by the *apply_inv_qft* method with following arguments:

* Quantum program with the first part of the algorithm implemented
* Auxiliar Qbits for applying the $\mathcal{QFT}^{\dagger}$

In [ ]:
prog = qft_pe.apply_inv_qft(prog, qft_pe.q_aux)

Now we can plot the circuit we have unitl the moment

In [ ]:
c = prog.to_circ()
%qatdisplay c --svg

####  Complete algorithm

The complete algorithm is executed by the *apply_pe_wqft* that need as arguments.

* Quantum Program with initial_state
* Quantum Routine or AbstractGate with phase operator $\mathcal{P}$
* Auxiliars Qbits


In [ ]:
prog = qft_pe.apply_pe_wqft(qft_pe.q_prog, qft_pe.q_gate, qft_pe.q_aux)

In [ ]:
c = prog.to_circ()
%qatdisplay c --svg

#### 2.2.3. classical QPE Algorithm execution

Once the QLM program is constructed the alogrithm should be executed. For this the *run_qprogram* method from the class allow to execute it. Following arguments should  be provided:

* q_prog : with the complete *CQPE* algorithm
* q_aux : auxiliar qbits 
* shots 
* linalg_qpu: QLM solver

This method creates the circuit, the asociated job and execute it. The returns of this method are:

* results : DataFrame with the results of the simulation.
* circuit : QLM circuit

In [ ]:
results, circuit = qft_pe.run_qprogram(prog, qft_pe.q_aux, 100, linalg_qpu=linalg_qpu)

In [ ]:
results

The results DataFrame has the following columns: 

* **States**: Posible quantum states of the auxiliar qbits
* **Int_lsb**:  conversion from the quantum state to a integer following **lsb** (bit fathest to the right will be least significant)
* **Probability**: Computed frequency of the quantum state.
* **Int**: conversion from the quantum state to a integer (bit fathest to the right will be most significant)
* **Phi**: is the estimated obtained phase and it is computed as: $\frac{Int}{2^{m}}$ where $m$ is the number of qbits used for phase estimation


In [ ]:
%qatdisplay circuit --svg

From *results* pdf the important column is **Phi**. Following the Qiskit example this column is $\varphi$ and the searched phase is: $\phi=2\pi\varphi$. In the qiskit example $\varphi=0.25$. 

We can sort the DataFrame by the **Probability** column and the desired $\phi$ will be the most probable one that should be the Qiskit result: $\varphi=0.25$ (this can be seen in folowing cell)

In [ ]:
results.sort_values('Probability', ascending=False).iloc[0]['Phi']

#### 2.2.4. CQPE: post proccessing

Typically the phase in radians is provide in this kind of phase estimation problems. For getting this result we use the *post_proccess*. The *results* DataFrame shold be provided and the output will be another DataFrame where folowing columns were add to the input DataFrame:

* **Phi**: is the estimated obtained phase and it is computed as: $\frac{BitInt}{2^{c_b}}$ where $c_b$ is the number of classical bits 
* **2*theta**: this is the phase of the unitary operator in radians:$2\theta = 2\pi\varphi$. In this case we calculate the phase of the unitary operator as the double of an angle $\theta$.
* **theta**: this is the halve of the phase of the unitary operator: $\theta = \pi\varphi$
* **theta_90**: is the $\theta$ between $(0, \frac{\pi}{2}$)

In this module our convention when $|\Psi\rangle $ is an eigenvalue of an unitary operator $\mathcal{Q}$ is:

$$\mathcal{Q}|\Psi\rangle = e^{2i\theta}|\Psi\rangle$$

In [ ]:
final_results = qft_pe.post_proccess(results)

In [ ]:
final_results

In [ ]:
final_results.sort_values('Probability', ascending=False)

### 2.3. Class CQPE: complete execution

In section 2.2 the complete algorithm step by step was explained. The methods shown in this sub section were shown only for pedagogical purpouses but users **SHOULD NOT** use these methods. Instead user **SHOULD USE** the **pe_qft** were all the steps explained in secion 2.2 are executed. When using this *method* following properties are populated:

* *final_results*: the DataFrame with the result of the *post_proccess* method.
* *results*: with the QLM results
* *circuit*: with the QLM circuit
* *post_proccess*: time with post proccess for the classical QPE algorithm (elapsed time of the 2.2.4 subsection)

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
pe_qft_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 100
}
qft_pe = CQPE(**pe_qft_dict)

In [ ]:
qft_pe.pe_qft()

In [ ]:
qft_pe.final_results

In [ ]:
qft_pe.final_results.sort_values('Probability', ascending = False)

In [ ]:
qft_pe.final_results.iloc[
    qft_pe.final_results['Probability'].idxmax()
]['Phi']

In [ ]:
qft_pe.time_qpe_post_procces

## Another example

We are going to reproduce now the qiskit example under the section *IPE example with a 2-qubit gate* (in https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb). 

In this case they use 2 qbits and want to estimate the phase for a unitary operator $\mathcal{cT}$ operator. this operator adds a $\frac{\pi}{4}$ phase to state $|11\rangle$ and leave unchanged other states. 


In [ ]:
#create initial state
n_qbits = 2
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

In [ ]:
#Create cT operator
unitary_operator = qlm.QRoutine()
uq_qbits = unitary_operator.new_wires(n_qbits)
unitary_operator.apply(qlm.PH(np.pi/4.0).ctrl(), 0, 1)

In [ ]:
%qatdisplay unitary_operator --svg

In [ ]:
#now PE with QFT!!
n_cbits = 3
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 100
}
qft_pe = CQPE(**qft_pe_dict)

In [ ]:
qft_pe.pe_qft()

In [ ]:
qft_pe.final_results.sort_values('Probability', ascending = False)

In [ ]:
qft_pe.time_qpe_post_procces

In [ ]:
qft_pe.time_pdf

The result for $\varphi=0.125$ that is the value obtained in the Qiskit example.

## 3. Application to Amplitude Estimation

The problem of *Amplitude Estimation* is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$.

In order to solve the *Amplitude Estimation* problem the classical *Phase Estimation* algorithm with *QFT* can be used. So we can take advantage of our **CQPE** class for doing this. This section explains the procedure.


First we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

Following cells creates the initial state $|\Psi\rangle$


In [ ]:
from QQuantLib.DL.data_loading import load_probability
from QQuantLib.AA.amplitude_amplification import grover

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

Next we will show how the Phase Estimation problem relates to the Amplitude Estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow |\Psi\rangle\\
    & \mathcal{P} \longrightarrow \mathcal{G}
    \end{array}
$$
The first equation means that, in the phase estimation context, the initial state is $|\Psi\rangle$ and the phase operator is $\mathcal{G}$, the Grover operator corresponding to our amplitude estimaiton problem. 

In the next cell we define the grover operator for our problem.  

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
grover_gate = grover(oracle, target, index)
#Comment before line and uncoment following for For multiplexor implementation of multi-controlled Z gate
#grover_gate = grover(oracle, target, index, mcz_qlm=False)
%qatdisplay grover_gate --depth 3 --svg

Here we have used that our target state $|1\rangle$ in binary representation is $001$. See notebook *02_AmplitudeAmplification_Operators* for more information about building Grover operators.

Now that we have translated our amplitude amplification probelm to an phase estimation problem we proceed to use our class normally. We provide the *oracle* as the **initial_state** and the correspondient Grover-like operator as the **unitary_operator**. Additionally the number of auxiliar_qbits_number (*auxiliar_qbits_number*) for estimating the phase should be provided.

In [ ]:
n_cbits = 6
#We create a python dictionary for configuration of class
qft_pe_dict = {
    'initial_state': oracle,
    'unitary_operator': grover_gate,
    'qpu' : linalg_qpu,
    'auxiliar_qbits_number' : n_cbits,  
    'shots': 100
}
qft_pe = CQPE(**qft_pe_dict)
qft_pe.pe_qft()

In [ ]:
qft_pe.final_results.sort_values('Probability', ascending = False)

In [ ]:
#We can plot the obtained the frequency vs the theta
qft_pe.final_results.sort_values('theta', inplace=True)
plt.plot(qft_pe.final_results['theta'], qft_pe.final_results['Probability'], 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel('Probability')

In [ ]:
#We can plot the obtained the frequency vs the theta in the [0, pì/2] range
qft_pe.final_results.sort_values('theta_90', inplace=True)
plt.plot(qft_pe.final_results['theta_90'], qft_pe.final_results['Probability'], 'o-')
plt.xlabel(r'$\theta$')
plt.ylabel('Probability')
plt.title(r'$\theta \in [0, \frac{\pi}{2}]$')

Last, we use the mapping $a = \cos(\theta)^2$ to obtain the result of our amplitude estimation problem.

In [ ]:
qft_pe.final_results['P_Psi_1']=np.cos(qft_pe.final_results['theta_90'])**2

In [ ]:
qft_pe.final_results.sort_values('Probability', inplace=True, ascending=False)

In [ ]:
qft_pe.final_results

In [ ]:
print("Classical result: ",probability[1])
print("Quantum result: ",qft_pe.final_results['P_Psi_1'].iloc[0])

In order to improve the Quantum results more qbits for estimation should be added!

In [ ]:
print('Test OK: ', probability[1]- qft_pe.final_results['P_Psi_1'].iloc[0] < 0.005)

## 4. Amplitude Estimation with classical Quantum Phase Estimation

In order to simplify **Amplitude Estimation** calculations done with the **CQPE** class (see Section 3) the **CQPEAE** (stands for **C**lassical **Q**uantum **P**hase **E**stimation **A**mplitude **Estimation**) class was created. The idea behind this class is dealing wiht the initial overheating showed in Section 3 for using **CQPE** in a straightforward way for *Amplitude Estimation* calculations. The class is under the module **ae_classical_qpe.py** of the *Amplitude Estimation* package of the *QQuantLib* library  (**QQuantLib/AE/ae_classical_qpe**)

**Note**

The idea is that the **CQPEAE** have a similar scheme like the **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae** module

In order to create **CQPEAE** asociated object following inputs are mandatory:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binnary representation as a python list
3. index: list of the qbits affected by the Grover operator.

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
#Here we created the mandatory oracle
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

#This will be the target state for grover and the list of qbits affected by Grover operator
target = [0, 0, 1]
index = range(oracle.arity)

Now we have all mandatory inputs so we created the *CQPEAE* object. Following **MLAE class** convention other parameters can be provided with a python dictionary. In this case the following keys can be provided:

* *auxiliar_qbits_number*: number of qbits for doing phase estimation (default: 8)
* *shots*: number of shots (default: 100)
* *qpu*: qpu solver
* mcz_qlm: for using QLM multi-controlled Z gate (True, default) or using multiplexor implementation (False)

In [ ]:
from QQuantLib.AE.ae_classical_qpe import CQPEAE

In [ ]:
ae_pe_qft_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 4,
    'shots': 10,
    'mcz_qlm': True    
}

ae_pe_qft = CQPEAE(
    oracle=oracle,
    target=target,
    index=index, 
    **ae_pe_qft_dict
)

When instantiated the *CQPEAE* class the *Grover* operator asociated to the oracle operator is created. It can be access using atribute *_grover_oracle*.

In [ ]:
grover_circ = ae_pe_qft._grover_oracle
%qatdisplay grover_circ --depth 3 --svg

Now the *ae_pe_qft* object have all mandatory inputs for calling the *CQPE* class and executing the algorithm. The **run** method from **CQPEAE** class do this steps. Additionally when the *run* method is executed following porperties from **CQPEAE** class are populated:

* *cqpe*: this property is an object from the **CQPE** class that was initalized with corresponding arguments created by the **CQPEAE** class.
* *final_results*: this is the final_results obtained from the *pe_qft* method of the property object *cqpe* (see section 2.3).
* *theta*: estimated $\theta$ obtained from **CQPE** algorithm
* *ae* amplitude estimated solution based on  $a=cos^2 \theta$
* *run_time*: elapsed time of a complete execution of the **run** method



In [ ]:
a_estimated = ae_pe_qft.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_pe_qft.final_results)
print('ae_pe_withQFT.theta= ', ae_pe_qft.theta)
print('ae_pe_withQFT.a= ', ae_pe_qft.ae)
print("Classical result: ",probability[1])

In [ ]:
print('Test OK: ',np.abs(ae_pe_qft.ae-probability[1])< 0.005)

In [ ]:
print('Elapsed time for run method: ',ae_pe_qft.run_time)

Of course the configuration of the algorithm can be changed by seting the atributes of the class an calling the *run* mehtod again!!

In [ ]:
ae_pe_qft.auxiliar_qbits_number = 10
print(ae_pe_qft.auxiliar_qbits_number)

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_pe_qft.theta)
print('ae_pe_withQFT.a= ', ae_pe_qft.ae)
print("Classical result: ",probability[1])

In [ ]:
print('Elapsed time for run method: ',ae_pe_qft.run_time)

As explained before, **cqpe** atribute from *CQPEAE* class is an object created from class **CQPE** so all the methods and attributes of this class can be accesed using the **cqpe** atribute (**NOT RECOMMENDED**).

We can acces to the quantum circuit used by the **CQPE**

In [ ]:
#First we access to the atribute tha stores the CQPE object
attribute_pewqtf = ae_pe_qft.cqpe
#Then we take the circuit attribute from the PhaseEstimationwQFT object (see above in the notebook)
c=attribute_pewqtf.circuit
%qatdisplay c --svg --depth 0

In [ ]:
ae_pe_qft.cqpe.time_pdf

In [ ]:
ae_pe_qft.cqpe.time_qpe_post_procces

In [ ]:
grover_oracle = ae_pe_qft._grover_oracle
%qatdisplay grover_oracle --depth 3